# Experiment 005: Comprehensive Ensemble from ALL Sources

The evaluator identified that we haven't explored all pre-optimized sources.
There are 692 CSV files in the snapshots directory!

This experiment will:
1. Scan ALL CSV files
2. Score each per-N
3. Build ensemble picking best per-N from ALL sources
4. Compare to current best

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import os
import glob
from tqdm import tqdm
import shutil

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    return max(all_points.max(axis=0) - all_points.min(axis=0))

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf'), None
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n, trees

def compute_total_score(df):
    return sum(compute_score_for_n(df, n)[0] for n in range(1, 201))

print("Functions defined")

Functions defined


In [2]:
# Find ALL CSV files in snapshots
all_csvs = glob.glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f"Found {len(all_csvs)} CSV files")

# Also add external data CSVs
external_csvs = glob.glob('/home/code/external_data/**/*.csv', recursive=True)
all_csvs.extend(external_csvs)
print(f"Total with external data: {len(all_csvs)} CSV files")

Found 692 CSV files
Total with external data: 699 CSV files


In [3]:
# Load and validate each CSV, keeping only valid submission files
valid_sources = []

for csv_path in tqdm(all_csvs, desc="Scanning CSVs"):
    try:
        df = pd.read_csv(csv_path)
        # Check if it's a valid submission file
        if 'id' in df.columns and 'x' in df.columns and 'y' in df.columns and 'deg' in df.columns:
            # Check if it has the right number of rows (20100 for complete submission)
            if len(df) >= 20000:  # Allow some flexibility
                valid_sources.append(csv_path)
    except Exception as e:
        pass

print(f"\nFound {len(valid_sources)} valid submission files")

Scanning CSVs:   0%|          | 0/699 [00:00<?, ?it/s]

Scanning CSVs:   1%|          | 5/699 [00:00<00:14, 47.40it/s]

Scanning CSVs:   2%|▏         | 11/699 [00:00<00:13, 51.36it/s]

Scanning CSVs:   2%|▏         | 17/699 [00:00<00:12, 52.63it/s]

Scanning CSVs:   3%|▎         | 23/699 [00:00<00:12, 53.34it/s]

...

Scanning CSVs:  98%|█████████▊| 684/699 [00:10<00:00, 61.80it/s]

Scanning CSVs:  99%|█████████▉| 691/699 [00:10<00:00, 59.56it/s]

Scanning CSVs: 100%|█████████▉| 698/699 [00:11<00:00, 58.35it/s]

Scanning CSVs: 100%|██████████| 699/699 [00:11<00:00, 63.35it/s]


Found 695 valid submission files


In [4]:
# Score each valid source and find total score
print("Scoring each source...")
source_scores = []

for csv_path in tqdm(valid_sources, desc="Scoring sources"):
    try:
        df = pd.read_csv(csv_path)
        total_score = compute_total_score(df)
        if total_score < 200:  # Sanity check - valid scores should be < 200
            source_scores.append((csv_path, total_score))
    except Exception as e:
        pass

# Sort by score
source_scores.sort(key=lambda x: x[1])

print(f"\nTop 20 sources by total score:")
for path, score in source_scores[:20]:
    print(f"  {score:.6f}: {path.split('/')[-1]}")

Scoring each source...


Scoring sources:   0%|          | 0/695 [00:00<?, ?it/s]

Scoring sources:   0%|          | 1/695 [00:01<23:02,  1.99s/it]

Scoring sources:   0%|          | 2/695 [00:03<22:56,  1.99s/it]

Scoring sources:   0%|          | 3/695 [00:05<22:48,  1.98s/it]

...

Scoring sources: 100%|█████████▉| 693/695 [22:43<00:03,  1.97s/it]

Scoring sources: 100%|█████████▉| 694/695 [22:45<00:01,  1.97s/it]

Scoring sources: 100%|██████████| 695/695 [22:47<00:00,  1.97s/it]

Scoring sources: 100%|██████████| 695/695 [22:47<00:00,  1.97s/it]


Top 20 sources by total score:
  51.663965: submission_opt.csv
  67.727119: submission_v21.csv
  67.772662: submission.csv
  67.772662: ensemble_submission.csv
  70.623733: submission_v21.csv
  70.629851: submission.csv
  70.659959: submission.csv
  70.659959: submission_best.csv
  70.659959: candidate_000.csv
  70.659959: candidate_002.csv
  70.659959: candidate_003.csv
  70.659959: candidate_001.csv
  70.659959: santa-2025.csv
  70.659959: santa-2025.csv
  70.659959: submission.csv
  70.659959: santa-2025.csv
  70.659959: santa-2025.csv
  70.659982: submission.csv
  70.659982: submission.csv
  70.675510: corner_extraction.csv


In [5]:
# Build comprehensive ensemble: for each N, find the best source
print("\nBuilding comprehensive ensemble...")

# Load all valid dataframes
all_dfs = {}
for path, score in tqdm(source_scores, desc="Loading dataframes"):
    try:
        all_dfs[path] = pd.read_csv(path)
    except:
        pass

print(f"Loaded {len(all_dfs)} dataframes")


Building comprehensive ensemble...


Loading dataframes:   0%|          | 0/683 [00:00<?, ?it/s]

Loading dataframes:   1%|          | 5/683 [00:00<00:13, 48.79it/s]

Loading dataframes:   1%|▏         | 10/683 [00:00<00:14, 45.70it/s]

Loading dataframes:   2%|▏         | 15/683 [00:00<00:14, 44.88it/s]

...

Loading dataframes:  96%|█████████▌| 656/683 [00:11<00:00, 90.06it/s]

Loading dataframes:  98%|█████████▊| 667/683 [00:11<00:00, 94.68it/s]

Loading dataframes:  99%|█████████▉| 678/683 [00:11<00:00, 98.50it/s]

Loading dataframes: 100%|██████████| 683/683 [00:11<00:00, 57.66it/s]

Loaded 683 dataframes


In [6]:
# For each N, find the best source
best_per_n = {}  # n -> (score, source_path, trees_df)

for n in tqdm(range(1, 201), desc="Finding best per N"):
    best_score = float('inf')
    best_source = None
    best_trees = None
    
    for path, df in all_dfs.items():
        score, trees = compute_score_for_n(df, n)
        if score < best_score:
            best_score = score
            best_source = path
            best_trees = trees
    
    best_per_n[n] = (best_score, best_source, best_trees)

# Compute ensemble total
ensemble_total = sum(best_per_n[n][0] for n in range(1, 201))
print(f"\nEnsemble total score: {ensemble_total:.6f}")

Finding best per N:   0%|          | 0/200 [00:00<?, ?it/s]

Finding best per N:   0%|          | 1/200 [00:01<05:24,  1.63s/it]

Finding best per N:   1%|          | 2/200 [00:03<05:12,  1.58s/it]

Finding best per N:   2%|▏         | 3/200 [00:04<05:10,  1.58s/it]

Finding best per N:   2%|▏         | 4/200 [00:06<05:13,  1.60s/it]

...

Finding best per N:  99%|█████████▉| 198/200 [21:54<00:23, 11.66s/it]

Finding best per N: 100%|█████████▉| 199/200 [22:05<00:11, 11.71s/it]

Finding best per N: 100%|██████████| 200/200 [22:17<00:00, 11.75s/it]

Finding best per N: 100%|██████████| 200/200 [22:17<00:00,  6.69s/it]


Ensemble total score: 51.423527


In [7]:
# Analyze which sources win for which N values
source_wins = {}
for n in range(1, 201):
    source = best_per_n[n][1]
    source_name = source.split('/')[-1] if source else 'None'
    source_wins[source_name] = source_wins.get(source_name, 0) + 1

print("Source wins distribution:")
for source, wins in sorted(source_wins.items(), key=lambda x: -x[1]):
    print(f"  {source}: {wins} N values")

Source wins distribution:
  submission_opt.csv: 151 N values
  submission.csv: 26 N values
  submission_v21.csv: 18 N values
  submission_best.csv: 5 N values


In [8]:
# Compare to current best
df_baseline = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
baseline_total = compute_total_score(df_baseline)

print(f"\nComparison:")
print(f"Baseline (saspav): {baseline_total:.6f}")
print(f"Comprehensive ensemble: {ensemble_total:.6f}")
print(f"Improvement: {baseline_total - ensemble_total:.9f}")

# Show N values where ensemble is different from baseline
print("\nN values where ensemble differs from baseline:")
for n in range(1, 201):
    baseline_score, _ = compute_score_for_n(df_baseline, n)
    ensemble_score = best_per_n[n][0]
    if abs(ensemble_score - baseline_score) > 1e-9:
        source_name = best_per_n[n][1].split('/')[-1] if best_per_n[n][1] else 'None'
        print(f"  N={n}: baseline={baseline_score:.6f}, ensemble={ensemble_score:.6f}, diff={ensemble_score-baseline_score:.9f}, source={source_name}")


Comparison:
Baseline (saspav): 70.659959
Comprehensive ensemble: 51.423527
Improvement: 19.236432494

N values where ensemble differs from baseline:
  N=2: baseline=0.450779, ensemble=0.437328, diff=-0.013451604, source=submission_v21.csv
  N=3: baseline=0.434745, ensemble=0.296773, diff=-0.137972569, source=submission_v21.csv
  N=4: baseline=0.416545, ensemble=0.227236, diff=-0.189308474, source=submission_v21.csv
  N=5: baseline=0.416850, ensemble=0.212694, diff=-0.204155253, source=submission_v21.csv
  N=6: baseline=0.399610, ensemble=0.173625, diff=-0.225984939, source=submission_v21.csv
  N=7: baseline=0.399897, ensemble=0.157468, diff=-0.242428623, source=submission_v21.csv
  N=8: baseline=0.385407, ensemble=0.136883, diff=-0.248524298, source=submission_opt.csv
  N=9: baseline=0.387415, ensemble=0.128142, diff=-0.259273470, source=submission_opt.csv
  N=10: baseline=0.376630, ensemble=0.154792, diff=-0.221837680, source=submission_opt.csv
  N=11: baseline=0.375736, ensemble=0.1

  N=50: baseline=0.360753, ensemble=0.034288, diff=-0.326465576, source=submission_opt.csv
  N=51: baseline=0.365574, ensemble=0.102116, diff=-0.263457667, source=submission_opt.csv
  N=52: baseline=0.361339, ensemble=0.102543, diff=-0.258796538, source=submission_opt.csv
  N=53: baseline=0.361855, ensemble=0.047860, diff=-0.313995243, source=submission_opt.csv
  N=54: baseline=0.359200, ensemble=0.207795, diff=-0.151405244, source=submission_opt.csv
  N=55: baseline=0.355023, ensemble=0.253140, diff=-0.101883256, source=submission_opt.csv
  N=56: baseline=0.352281, ensemble=0.302045, diff=-0.050235372, source=submission_opt.csv
  N=57: baseline=0.354108, ensemble=0.231956, diff=-0.122152500, source=submission_opt.csv
  N=58: baseline=0.363488, ensemble=0.166537, diff=-0.196950799, source=submission_opt.csv
  N=59: baseline=0.360910, ensemble=0.169942, diff=-0.190967729, source=submission_opt.csv
  N=60: baseline=0.357258, ensemble=0.199067, diff=-0.158191452, source=submission_opt.csv

  N=79: baseline=0.348564, ensemble=0.319061, diff=-0.029503276, source=submission_opt.csv
  N=80: baseline=0.344880, ensemble=0.320408, diff=-0.024472723, source=submission_opt.csv
  N=81: baseline=0.355287, ensemble=0.217164, diff=-0.138122452, source=submission_opt.csv
  N=82: baseline=0.351531, ensemble=0.277738, diff=-0.073792992, source=submission_opt.csv
  N=83: baseline=0.352426, ensemble=0.251875, diff=-0.100550603, source=submission_opt.csv
  N=84: baseline=0.350726, ensemble=0.350706, diff=-0.000019525, source=submission_opt.csv
  N=85: baseline=0.350941, ensemble=0.304863, diff=-0.046078353, source=submission_opt.csv
  N=86: baseline=0.350088, ensemble=0.302809, diff=-0.047278518, source=submission_opt.csv
  N=87: baseline=0.353728, ensemble=0.350990, diff=-0.002737551, source=submission_opt.csv
  N=88: baseline=0.350671, ensemble=0.344050, diff=-0.006621005, source=submission_opt.csv
  N=89: baseline=0.346911, ensemble=0.315922, diff=-0.030988658, source=submission_opt.csv

  N=102: baseline=0.347140, ensemble=0.346901, diff=-0.000238745, source=submission_opt.csv
  N=103: baseline=0.344118, ensemble=0.301282, diff=-0.042835950, source=submission_opt.csv
  N=104: baseline=0.340810, ensemble=0.340400, diff=-0.000409795, source=submission_opt.csv
  N=105: baseline=0.342872, ensemble=0.342124, diff=-0.000747829, source=submission_opt.csv
  N=106: baseline=0.341375, ensemble=0.339308, diff=-0.002067700, source=submission_opt.csv
  N=107: baseline=0.346089, ensemble=0.345477, diff=-0.000611834, source=submission_opt.csv
  N=108: baseline=0.343559, ensemble=0.339675, diff=-0.003884809, source=submission_opt.csv
  N=109: baseline=0.340541, ensemble=0.338118, diff=-0.002422901, source=submission_opt.csv
  N=110: baseline=0.337603, ensemble=0.334906, diff=-0.002696694, source=submission_opt.csv
  N=111: baseline=0.343923, ensemble=0.279802, diff=-0.064121092, source=submission_opt.csv
  N=112: baseline=0.346450, ensemble=0.313568, diff=-0.032882091, source=submiss

  N=121: baseline=0.351327, ensemble=0.350965, diff=-0.000362448, source=submission_opt.csv
  N=122: baseline=0.350173, ensemble=0.348272, diff=-0.001901537, source=submission_opt.csv
  N=123: baseline=0.347904, ensemble=0.347850, diff=-0.000054617, source=submission_opt.csv
  N=124: baseline=0.346839, ensemble=0.346694, diff=-0.000144838, source=submission_v21.csv
  N=125: baseline=0.345548, ensemble=0.345203, diff=-0.000344990, source=submission_v21.csv
  N=126: baseline=0.344131, ensemble=0.323445, diff=-0.020686019, source=submission_opt.csv
  N=127: baseline=0.341912, ensemble=0.302127, diff=-0.039785596, source=submission_opt.csv
  N=128: baseline=0.343731, ensemble=0.343235, diff=-0.000495959, source=submission_opt.csv
  N=129: baseline=0.341387, ensemble=0.340801, diff=-0.000585828, source=submission_opt.csv
  N=130: baseline=0.338794, ensemble=0.338368, diff=-0.000426564, source=submission_v21.csv
  N=131: baseline=0.336296, ensemble=0.334001, diff=-0.002295528, source=submiss

  N=138: baseline=0.341028, ensemble=0.312215, diff=-0.028813126, source=submission_opt.csv
  N=139: baseline=0.340978, ensemble=0.321858, diff=-0.019120530, source=submission_opt.csv
  N=140: baseline=0.340098, ensemble=0.339209, diff=-0.000889720, source=submission_v21.csv
  N=142: baseline=0.341148, ensemble=0.334960, diff=-0.006187450, source=submission_opt.csv
  N=145: baseline=0.342839, ensemble=0.336726, diff=-0.006112907, source=submission_opt.csv
  N=146: baseline=0.343437, ensemble=0.318330, diff=-0.025107206, source=submission_opt.csv
  N=150: baseline=0.337064, ensemble=0.337024, diff=-0.000039956, source=submission_opt.csv
  N=151: baseline=0.340260, ensemble=0.339678, diff=-0.000582336, source=submission_opt.csv
  N=152: baseline=0.338486, ensemble=0.338476, diff=-0.000010831, source=submission_opt.csv
  N=153: baseline=0.336286, ensemble=0.336243, diff=-0.000043652, source=submission_opt.csv


  N=154: baseline=0.334158, ensemble=0.333939, diff=-0.000218969, source=submission_opt.csv
  N=155: baseline=0.332074, ensemble=0.331610, diff=-0.000463479, source=submission_opt.csv
  N=156: baseline=0.329987, ensemble=0.329912, diff=-0.000074934, source=submission_v21.csv
  N=157: baseline=0.341371, ensemble=0.329358, diff=-0.012013248, source=submission_opt.csv
  N=158: baseline=0.341273, ensemble=0.271440, diff=-0.069833199, source=submission_opt.csv
  N=159: baseline=0.340900, ensemble=0.308739, diff=-0.032160960, source=submission_opt.csv
  N=160: baseline=0.339400, ensemble=0.332823, diff=-0.006577021, source=submission_opt.csv
  N=161: baseline=0.338630, ensemble=0.337533, diff=-0.001097151, source=submission_opt.csv
  N=163: baseline=0.337516, ensemble=0.323625, diff=-0.013890841, source=submission_opt.csv
  N=164: baseline=0.337328, ensemble=0.333232, diff=-0.004095974, source=submission_opt.csv
  N=165: baseline=0.335569, ensemble=0.333502, diff=-0.002067067, source=submiss

  N=168: baseline=0.332475, ensemble=0.331724, diff=-0.000751069, source=submission_v21.csv
  N=169: baseline=0.342500, ensemble=0.341687, diff=-0.000812648, source=submission_opt.csv
  N=175: baseline=0.338682, ensemble=0.281514, diff=-0.057167906, source=submission_opt.csv
  N=176: baseline=0.338206, ensemble=0.337908, diff=-0.000298191, source=submission_opt.csv
  N=177: baseline=0.336326, ensemble=0.333007, diff=-0.003319239, source=submission_opt.csv
  N=178: baseline=0.334442, ensemble=0.334441, diff=-0.000000970, source=submission_opt.csv
  N=179: baseline=0.332595, ensemble=0.332571, diff=-0.000023415, source=submission_v21.csv
  N=180: baseline=0.331001, ensemble=0.330767, diff=-0.000234248, source=submission_opt.csv


  N=181: baseline=0.329946, ensemble=0.329763, diff=-0.000183171, source=submission_opt.csv
  N=185: baseline=0.338562, ensemble=0.338265, diff=-0.000296874, source=submission_v21.csv
  N=189: baseline=0.338821, ensemble=0.318436, diff=-0.020385311, source=submission_opt.csv
  N=190: baseline=0.338231, ensemble=0.337750, diff=-0.000481005, source=submission_opt.csv
  N=191: baseline=0.336758, ensemble=0.336430, diff=-0.000328295, source=submission_v21.csv
  N=192: baseline=0.335301, ensemble=0.334985, diff=-0.000315526, source=submission_v21.csv


In [9]:
# Save the ensemble submission
ensemble_rows = []
for n in range(1, 201):
    trees = best_per_n[n][2]
    if trees is not None:
        for _, row in trees.iterrows():
            ensemble_rows.append(row.to_dict())

ensemble_df = pd.DataFrame(ensemble_rows)
ensemble_df.to_csv('/home/submission/submission.csv', index=False)
print(f"Saved ensemble with {len(ensemble_df)} rows")

# Verify
df_verify = pd.read_csv('/home/submission/submission.csv')
verify_score = compute_total_score(df_verify)
print(f"Verified ensemble score: {verify_score:.6f}")

Saved ensemble with 20100 rows


Verified ensemble score: 51.423527


In [10]:
# Summary
print("="*60)
print("EXPERIMENT 005 SUMMARY: Comprehensive Ensemble")
print("="*60)
print(f"Total CSV files scanned: {len(all_csvs)}")
print(f"Valid submission files: {len(valid_sources)}")
print(f"Sources with valid scores: {len(source_scores)}")
print(f"\nBest single source: {source_scores[0][1]:.6f} ({source_scores[0][0].split('/')[-1]})")
print(f"Comprehensive ensemble: {verify_score:.6f}")
print(f"Improvement over best single: {source_scores[0][1] - verify_score:.9f}")
print("="*60)

EXPERIMENT 005 SUMMARY: Comprehensive Ensemble
Total CSV files scanned: 699
Valid submission files: 695
Sources with valid scores: 683

Best single source: 51.663965 (submission_opt.csv)
Comprehensive ensemble: 51.423527
Improvement over best single: 0.240437776


In [11]:
# Model wrapper for submission
class ComprehensiveEnsemble:
    def __init__(self, data='single'):
        self.data = data
        
    def load_best(self):
        return pd.read_csv('/home/submission/submission.csv')
    
    def save_submission(self, path):
        df = self.load_best()
        df.to_csv(path, index=False)
        return df

model = ComprehensiveEnsemble(data='single')
print("Model wrapper defined")

Model wrapper defined
